## Дипломный проект на курсе Data Science школа SkillFactory
# «Модель прогнозирования стоимости жилья для агентства недвижимости»

**Цель:** разработать модель, которая позволила бы агентству недвижимости обойти конкурентов по скорости и качеству совершения сделок.

**Поставленные задачи:**
1. Провести разведывательный анализ и очистку исходных данных. Обратите внимание, что данные в таблице реальные: в результате во многих признаках присутствуют дублирующиеся категории, ошибки ввода, жаргонные сокращения и т .д. Вам предстоит отыскать закономерности, самостоятельно расшифровать все сокращения, найти синонимы в данных, обработать пропуски и удалить выбросы. 
2. Выделить наиболее значимые факторы, влияющие на стоимость недвижимости.
3. Построить модель для прогнозирования стоимости недвижимости.
4. Разработать небольшой веб-сервис, на вход которому поступают данные о некоторой выставленной на продажу недвижимости, а сервис прогнозирует его стоимость.

**Описание данных:**<br>
<br>
➔ 'status' — статус продажи;<br>
➔ 'private pool' и 'PrivatePool' — наличие собственного бассейна;<br>
➔ 'propertyType' — тип объекта недвижимости;<br>
➔ 'street' — адрес объекта;<br>
➔ 'baths' — количество ванных комнат;<br>
➔ 'homeFacts' — сведения о строительстве объекта (содержит несколько
типов сведений, влияющих на оценку объекта);<br>
➔ 'fireplace' — наличие камина;<br>
➔ 'city' — город;<br>
➔ 'schools' — сведения о школах в районе;<br>
➔ 'sqft' — площадь в футах;<br>
➔ 'zipcode' — почтовый индекс;<br>
➔ 'beds' — количество спален;<br>
➔ 'state' — штат;<br>
➔ 'stories' — количество этажей;<br>
➔ 'mls-id' и 'MlsId' — идентификатор MLS (Multiple Listing Service, система
мультилистинга);<br><br>

➔ **'target'** — цена объекта недвижимости (целевой признак, который
необходимо спрогнозировать).

In [3]:
import numpy as np 
import pandas as pd

# импортируем библиотеки для визуализации
import matplotlib.pyplot as plt
import seaborn as sns 

# импортируем модуль регулярных выражений
import re

In [4]:
# зафиусируем RANDOM_SEED
RANDOM_SEED = 42

In [5]:
# зафиксируем версию пакетов
!pip freeze > requirements.txt

## 1. Знакомство с данными, первичная обработка

In [6]:
df = pd.read_csv('data/data.csv')
df.head()

,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
0,Active,NaN,Single Family Home,240 Heather Ln,3.5,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",Gas Logs,Southern Pines,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",2900,28387,4,NC,NaN,NaN,NaN,611019,"$418,000"
1,for sale,NaN,single-family home,12911 E Heroy Ave,3 Baths,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,Spokane Valley,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...","1,947 sqft",99216,3 Beds,WA,2.0,NaN,NaN,201916904,"$310,000"
2,for sale,NaN,single-family home,2005 Westridge Rd,2 Baths,"{'atAGlanceFacts': [{'factValue': '1961', 'fac...",yes,Los Angeles,"[{'rating': ['8/10', '4/10', '8/10'], 'data': ...","3,000 sqft",90049,3 Beds,CA,1.0,NaN,yes,FR19221027,"$2,895,000"
3,for sale,NaN,single-family home,4311 Livingston Ave,8 Baths,"{'atAGlanceFacts': [{'factValue': '2006', 'fac...",yes,Dallas,"[{'rating': ['9/10', '9/10', '10/10', '9/10'],...","6,457 sqft",75205,5 Beds,TX,3.0,NaN,NaN,14191809,"$2,395,000"
4,for sale,NaN,lot/land,1524 Kiscoe St,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Palm Bay,"[{'rating': ['4/10', '5/10', '5/10'], 'data': ...",NaN,32908,NaN,FL,NaN,NaN,NaN,861745,"$5,000"


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377185 entries, 0 to 377184
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   status        337267 non-null  object
 1   private pool  4181 non-null    object
 2   propertyType  342452 non-null  object
 3   street        377183 non-null  object
 4   baths         270847 non-null  object
 5   homeFacts     377185 non-null  object
 6   fireplace     103115 non-null  object
 7   city          377151 non-null  object
 8   schools       377185 non-null  object
 9   sqft          336608 non-null  object
 10  zipcode       377185 non-null  object
 11  beds          285903 non-null  object
 12  state         377185 non-null  object
 13  stories       226470 non-null  object
 14  mls-id        24942 non-null   object
 15  PrivatePool   40311 non-null   object
 16  MlsId         310305 non-null  object
 17  target        374704 non-null  object
dtypes: object(18)
memory usa

In [8]:
count_rows = df.shape[0]
print(f'Количество строк в датасете: {count_rows}')

Количество строк в датасете: 377185


В датасете 377185 строк и 17 признаков + столбец таргета. Все данные типа object. Почти во всех столбцах присутствуют пропуски.<br>
Сразу удалим полные дубликаты строк, если такие есть.

### Поиск и удаление дубликатов

In [9]:
#удаляем дубликаты
df = df.drop_duplicates(ignore_index=True)

# посчитаем сколько строк было удалено
duplicate_rows = count_rows - df.shape[0]
print(f'Удалено дубликатов: {duplicate_rows}')

Удалено дубликатов: 50


### Исследование пропусков

In [10]:
df.isnull().mean() * 100

status          10.584274
private pool    98.891378
propertyType     9.209699
street           0.000530
baths           28.188315
homeFacts        0.000000
fireplace       72.659127
city             0.009015
schools          0.000000
sqft            10.752118
zipcode          0.000000
beds            24.196640
state            0.000000
stories         39.952007
mls-id          93.386453
PrivatePool     89.311520
MlsId           17.730786
target           0.657589
dtype: float64

Признаки mls-id и MlsId являются внутренними риэлторскими метками, никак не влияющими на стоимость квартиры, поэтому данные столбцы можно удалить.

In [11]:
# Удаляем столбец PrivatePool
df = df.drop(['MlsId', 'mls-id'], axis=1)

### Столбцы private pool и PrivatePool, stories и fireplace

Самое большое количество пропусков в столбцах: PrivatePool и private pool, > 89%. Объединим данные столбцы, считая пропуски в них за отсутствие бассейна.

In [12]:
# посмотрим, какие варианты значений содержат столбцы
print(df['private pool'].unique())
print(df['PrivatePool'].unique())

# создаем третий столбец и заполняем его данными из двух, меняем данные на булевые переменные
df['private_pool_final'] = df['private pool'].fillna('') + df['PrivatePool'].fillna('')
df['private_pool_final'] = df['private_pool_final'].replace(['yes', 'Yes'], True)
df['private_pool_final'] = df['private_pool_final'].replace('', False)
# проеверим уникальные значения в новом столбце
df['private_pool_final'].unique()
# удаляем обработанные столбцы
df = df.drop(['private pool', 'PrivatePool'], axis=1)

[nan 'Yes']
[nan 'yes' 'Yes']


В столбце fireplace 73% пропусков, посмотрим на содержание столбца.

In [13]:
df['fireplace'].value_counts()

yes                                                                     50353
Yes                                                                     20856
1                                                                       14544
2                                                                        2432
Not Applicable                                                           1993
                                                                        ...  
Free-standing, Insert, Wood                                                 1
Wood Burning, Attached Fireplace Doors/Screen, Electric, Gas Starter        1
One, Living Room                                                            1
FAMILYRM, Great Room, Living Room                                           1
Ceiling Fan, SMAPL, Utility Connection, Walk-In Closets                     1
Name: fireplace, Length: 1653, dtype: int64

Изучив столбец наличия камина (fireplace), заметила, что в данный столбец писали информацию не только по каминам, но так же общую информацию по объекту недвижимости (такую как: наличие инженерных сетей, гардеробных и тд).Уникальных записей 1653. К сожалению, отфильтровать реальную информацию по наличию камина не представляется возможным, а пропуски в данном столбце очень большой (73%). Поэтому данный столбец удаляем. <br>
В столбце этажей (stories) 40% пропусков, так как определить на каком этаже квартира в доме затруднительно, а оставлять пропуски нельзя, придется удалить данный столбец тоже.

In [14]:
df = df.drop(['stories', 'fireplace'], axis=1)

### Пропуски и обработка столбцов beds, sqft, baths и city

Столбец beds

In [15]:
# переносим данные по площади в другие столбцы, они нам могут понадобиться для заполнения пропусков в столбце sqft
df['sqft_from_beds'] = df['beds'].str.extract(r'(\d+,\d+)\s+sqft', expand=False)
df['acres_from_beds'] = df['beds'].str.extract(r'([\d.]+)\s+acres', expand=False)

In [16]:
# удаляем лишние данные
df['beds'] = df['beds'].str.replace(r'(\d+,\d+)\s+sqft', '', regex=True)
df['beds'] = df['beds'].str.replace(r'([\d.]+)\s+acres', '', regex=True)

# точечно изменяем некоторые значения
df['beds'] = df['beds'].replace(['1 Bath, 2 Bedrooms, Cable TV Available, Dining Room, Eat-In Kitchen, Living Room', '1 Bath, 2 Bedrooms, Living Room, Range/Oven, Refrigerator', '1 Bath, 2 Bedrooms', '1 Bath, 2 Bedrooms, Eat-In Kitchen, Living Room, Range/Oven, Refrigerator'], 2)
df['beds'] = df['beds'].replace(['1 Bath, 3 or More Bedrooms, Cable TV Available, Dining Room, Eat-In Kitchen, Living Room, Range/Oven, Refrigerator', '3 or More Bedrooms, Dining Room, Living Room, Range/Oven, Refrigerator', '3 or More Bedrooms', '2 Baths, 3 or More Bedrooms'], 3)
df['beds'] = df['beds'].replace(["Based on Redfin's St Johns data, we estimate the home's value is $360,731, which is 2.2% less than its current list price.", "Based on Redfin's Raleigh data, we estimate the home's value is $708,248, which is 1.2% more than its current list price.", '-- bd', '-- sqft', '4 sqft', '60 sqft', '1 acre', '840 sqft', '540 sqft', '100 sqft', '871 sqft', '640 sqft', '831 sqft', '448 sqft', '248 sqft', '# Bedrooms 1st Floor'], 0)

# переносим данные по спальням в другие столбецы
df['beds_final1'] = df['beds'].str.extract(r'(\d+)\s*(?:Beds|bd)', expand=False)
df['beds_final2'] = df['beds'].str.extract(r'(\d+\.\d+|\d+)')

# создаем итоговы столбец beds_final и заполняем его данными из двух, заменяем пропуски на 0
df['beds_final'] = df['beds_final1'].fillna(0)
df['beds_final'] = df['beds_final2'].fillna(0)

# проверяем на пропуски
beds_nulldata = df['beds_final'].isnull().mean()*100
print(f'Пропуски в столбце beds_final: {beds_nulldata} %')

# меняем тип данных
df['beds_final'] = df['beds_final'].astype(float)

# удаляем отработанные столбцы
df = df.drop(['beds_final1', 'beds_final2', 'beds'], axis=1)

Пропуски в столбце beds_final: 0.0 %


Столбец sqft

In [17]:
# посмотрим на содержание столбца и пропуски
sqft_isnulldata = df['sqft'].isnull().mean()*100
print(f'Количество пропущенных значений: {sqft_isnulldata:.2f} %')
df['sqft'].unique()[0:20]

Количество пропущенных значений: 10.75 %


array(['2900', '1,947 sqft', '3,000 sqft', '6,457 sqft', nan, '897 sqft',
       '1,507', '3588', '1,930', '1,300 sqft', '3,130', '2,839 sqft',
       'Total interior livable area: 1,820 sqft', '2,454', '2,203',
       '3,325', '3,080 sqft', '1,612 sqft', '1,731 sqft',
       'Total interior livable area: 5,266 sqft'], dtype=object)

In [18]:
# оставляем только числовое значение без ,
df['sqft'] = df['sqft'].str.replace(',', '').str.extract('(\d+)')

# посмотрим, сколько пропусков в столбце
sqft_dataisnull = df['sqft'].isnull().sum()
print(f'Количество пропущенных значений sqft: {sqft_dataisnull}')

Количество пропущенных значений sqft: 41370


Попробуем заполнить пропуски в столбце с площадью данными из столбца beds (sqft_from_beds и acres_from_beds)

In [19]:
# убираем лишние знаки
df['sqft_from_beds'] = df['sqft_from_beds'].replace(',', '', regex=True)

# заполняем столбец данными из sqft_from_beds
df['sqft'] = df['sqft'].fillna(df['sqft_from_beds'])

# проверим, сколько пропусков в столбце столо теперь
sqft_dataisnull_1 = df['sqft'].isnull().sum()
print(f'Количество пропущенных значений sqft после дополнения: {sqft_dataisnull_1}')

Количество пропущенных значений sqft после дополнения: 40049


In [20]:
# посмотрим содержание столбца
df['acres_from_beds'].unique()[0:500]

# переведем столбец в числовое значение и создадим новый столбец, где акры переведем в футы (1 акр = 43560 футов кв.)
df['acres_from_beds'] = df['acres_from_beds'].astype(float)
df['acres_from_beds_new'] = df['acres_from_beds'] * 43560

# заполняем столбец данными из sqft_from_beds
df['sqft'] = df['sqft'].fillna(df['acres_from_beds_new'])

# проверим, сколько пропусков в столбце столо теперь
sqft_dataisnull_2 = df['sqft'].isnull().sum()
print(f'Количество пропущенных значений sqft после второго дополнения: {sqft_dataisnull_2}')

Количество пропущенных значений sqft после второго дополнения: 38458


In [21]:
# заменяем пропуски на значение 0
df['sqft'] = df['sqft'].fillna(0)

# меняем тип данных
df['sqft'] = df['sqft'].astype(int)

# удаляем лишние столбцы
df = df.drop(['sqft_from_beds', 'acres_from_beds', 'acres_from_beds_new'], axis=1)

Столбец baths

In [22]:
# посмотрим на содержание столбца и пропуски
baths_isnulldata = df['baths'].isnull().mean()*100
print(f'Количество пропущенных значений: {baths_isnulldata:.2f} %')
df['baths'].unique()[0:20]

Количество пропущенных значений: 28.19 %


array(['3.5', '3 Baths', '2 Baths', '8 Baths', nan, '2', '3',
       'Bathrooms: 2', '1,750', '4 Baths', '2 ba', 'Bathrooms: 5',
       '1,000', '7 Baths', '2.0', '3.0', 'Bathrooms: 1', '4.0',
       '2.1 Baths', '2.5 Baths'], dtype=object)

In [23]:
# удалим из данных буквы и пробелы
df['baths'] = df['baths'].str.replace('[a-zA-Z+:]','', regex=True)

# точечно меняем некоторые данные
df['baths'] = df['baths'].replace(['~', '0 / 0', '. . ', '-- ', '—'], 0)
df['baths'] = df['baths'].replace(['1 / 1-0 / 1-0 / 1-0','1-0 / 1-0 / 1', '1 / 1 / 1 / 1', '1-2 ', '1,000'], 1)
df['baths'] = df['baths'].replace(['2-1 / 2-1 / 1-1 / 1-1', '2,000'], 2)
df['baths'] = df['baths'].replace(['3-1 / 2-2', '3,000'], 3)
df['baths'] = df['baths'].replace('116 / 116 / 116', 116)
df['baths'] = df['baths'].replace('7,500', 7.5)
df['baths'] = df['baths'].replace('5,000', 5)
df['baths'] = df['baths'].replace('3,500', 3.5)
df['baths'] = df['baths'].replace('2,250', 2.25)
df['baths'] = df['baths'].replace('1,250', 1.25)
df['baths'] = df['baths'].replace('2,500', 2.5)
df['baths'] = df['baths'].replace('2,750', 2.75)
df['baths'] = df['baths'].replace('4,000', 4)
df['baths'] = df['baths'].replace('1,750', 1.75)
df['baths'] = df['baths'].replace('1,500', 1.5)

# оставляем только числовое значение
df['baths'] = df['baths'].str.extract(r'(\d+\.\d+|\d+)')

Так как пропуски в данном столбце можно считать за отсутствие ванной, то заменяем пропуски на 0

In [24]:
# земеняем пропуски на 0
df['baths'] = df['baths'].fillna(0)

# меняем тип данных
df['baths'] = df['baths'].astype(float)

df['baths'].unique()[0:20]

array([  3.5,   3. ,   2. ,   8. ,   0. ,   4. ,   5. ,   7. ,   1. ,
         2.1,   2.5,   4.5,   6. ,   5.5,   1.5,   9. ,  12. , 750. ,
        10. ,  19. ])

Столбец city

In [25]:
# найдем пропущенные значения в столбце city
city_isnulldata = df['city'].isnull()

# найдем список индексов, у которых нет города
print(set(list(df[city_isnulldata]['zipcode'])))

{'38732', '33954', '78045', '34473', '34744', '32668', '77032', '34474', '32686', '34481', '33955', '34747', '32179', '34488', '20003', '34432', '33126', '34741'}


При формировании списка индексов, где пропущены города, оказалось, что таких пар всего 18, поэтому я решила вручную создать список таких пар (смотреть по индексу к какому городу принадлежит он) и заполнить пропуски по городам через индексы

In [26]:
zipcode_city_list = {'20003':'Washington', '32179':'Ocklawaha', '32668':'Morriston', '32686':'Reddick', '33126':'Miami', '33954':'Port Charlotte',
                     '33955':'Punta Gorda', '34432':'Dunnellon','34473':'Ocala', '34474':'Ocala', '34481':'Ocala', '34488':'Silver Springs',
                     '34741':'KISSIMMEE', '34744':'Kissimmee', '34747':'Kissimmee	', '38732':'Cleveland', '77032':'Houston', '78045':'Laredo'}

def find_city (zipcode):
    # проверяем, есть ли zipcode в словаре, и если есть, возвращаем соответствующее значение
    if zipcode in zipcode_city_list:
        return zipcode_city_list[zipcode]
    else:
        return None  # если zipcode не найден, возвращаем None

df['city_add'] = df['zipcode'].apply(find_city)

# применяем функцию к столбцу 'zipcode' и записываем результаты в столбец 'city_add'
df['city_add'] = df['zipcode'].apply(find_city)

# заполняем пропущенные данные в столбце city по столбцу city_add
df['city'].fillna(df['city_add'], inplace=True)

# проверяем остались ли пропущенные значения
city_isnulldata_new = df['city'].isnull().mean()*100
print(f'Количество пропущенных значений столбец city: {city_isnulldata_new:.2f} %')

# удаляем ненужный столбец
df = df.drop(['city_add'], axis=1)

Количество пропущенных значений столбец city: 0.00 %


### Пропуски и обработка столбца status и propertyType

Cтолбец status

In [27]:
# посмотрим содержание столбца
uniq_status_sum = df['status'].nunique()
print(f'Количество уникальных значений столбца status: {uniq_status_sum}')

df['status'].unique()[:50]

Количество уникальных значений столбца status: 159


array(['Active', 'for sale', nan, 'New construction', 'New', 'For sale',
       'Pending', 'P', 'Active/Contingent', 'Pre-foreclosure / auction',
       ' / auction', 'Under Contract', 'Under Contract   Showing',
       'Pre-foreclosure', 'Under Contract Backups', 'foreclosure',
       'Active Under Contract', 'Foreclosed', 'Option Pending',
       'Under Contract Show', 'for rent', 'Auction', 'A Active',
       'Contingent', 'Pending   Continue To Show', 'Price Change',
       'Back on Market', 'Active Option', 'Foreclosure', 'recently sold',
       'Coming soon: Nov 21.', 'Contingent Finance And Inspection',
       'Coming soon: Dec 4.', 'P Pending Sale', 'Coming soon: Nov 23.',
       'Active With Contingencies', 'Pending Ab', 'Pf', 'Contingent Show',
       'Contract P', 'Contingent Take Backup', 'Apartment for rent',
       'Backup Contract', 'Option Contract', 'Pending Continue To Show',
       'pending', 'Pending Inspection', 'Active Option Contract', 'C',
       'Auction - Acti

In [28]:
# смотрим сколько пропусков в столбце
df['status'].isnull().mean()*100

10.584273536001696

Очень много уникальных значений, но большинство значений можно систематизировать, сократив количество уникальных значений. Создадим словарь значений. А так же заменим пропуски на 'unknown'

In [29]:
values_status = {
    "For sale": ["for sale", "For sale", "New construction", "New"],
    "Active": [
        "Active", "A Active", "Active/Contingent", "Active Under Contract", "Active Option", "Auction - Active",
        "Active With Contingencies", "Active Option Contract", "Active Contingency", "Active Backup",
        "Active Contingent", "Active - Auction", "Active With Offer", "Active - Contingent", "Active with Contract",
        "Temporary Active", "Re Activated", "Reactivated"],
    "Pending": [
        "P", "Pending", "pending", "P Pending Sale", "Pending Ab", "Pending Continue To Show",
        "Pending Inspection", "Pending Offer Approval", "Pending In", "Pending W/Insp Finance", "Pending Fe",
        "Pending W/Backup Wanted", "Pending Backups Wanted", "Pending With Contingencies", "Lease/Purchase Pending",
        "Pending Bring Backup", "Pending - Taking Backups", "Pending - Continue to Show",
        "Pending Taking Backups", "Offer Pending Signature", "Pending (Do Not Show)", "Pending W/ Cont.",
        "Pending W/Escape Clause", "Pending - Backup Offer Requested", "Pending Sale"],
    "Contingent": [
        "Contingent", "Contingent Finance And Inspection", "Contingent Show",
        "Contingent Take Backup", "Contingent - Sale of Home", "Contingent Finance and Inspection",
        "C Continue Show", "Contingent   Show", "Contingent   Release", "Contingent   No Show",
        "CT Insp - Inspection Contingency", "Contingent   Foreclosure", "Conting Accpt Backups",
        "Contingent - Financing", "Contingency 48 Hr (+/ )", "Contingency Contract", "Contingent Escape"],
    "Under contract": [
        "Under Contract", "Under Contract   Showing", "Under Contract Backups", "Under Contract Show",
        "Under Contract - Show", "Under Contract - No Show", "Under contract", "U Under Contract",
        "Contract Contingent On Buyer Sale", "Contract P", "Ct", "Uc Continue To Show",
        "Under Contract Taking Back Up Offers", "Under Contract W/ Bckp", "Contract"],
    "For rent": ["for rent", "Apartment for rent", "Condo for rent"],
    "Auction": ["Auction", "Pre-foreclosure", "Pre-foreclosure / auction", "/ auction", "Foreclosed", "foreclosure", "Foreclosure"],
    "Due diligence": ["Due Diligence Period"],
    "Recently sold": ["recently sold"],
    "Price change": ["Price Change"],
    "Back on market": ["Back on Market", "Back On Market"],
    "Closed": ["Closed"],
    "Listing extended": ["Listing Extended"],
    "Coming soon": [
        "Coming soon: Nov 21.", "Coming soon: Dec 4.", "Coming soon: Nov 23.", "Coming soon: Nov 29.",
        "Coming soon: Dec 2.", "Coming soon: Dec 10.", "Coming soon: Dec 24.", "Coming soon: Nov 14.",
        "Coming soon: Nov 22.", "Coming soon: Oct 21.", "Coming soon: Dec 14.", "Coming soon: Oct 24.",
        "Coming soon: Dec 18.", "Coming soon: Dec 16.", "Coming soon: Dec 3.", "Coming soon: Dec 25.",
        "Coming soon: Nov 11.", "Coming soon: Nov 28.", "Coming soon: Nov 17.", "Coming soon: Dec 6.",
        "Coming soon: Nov 27.", "Coming soon: Nov 26.", "Coming soon: Dec 7.", "Coming soon: Dec 27.",
        "Coming soon: Dec 11.", "Coming soon: Dec 5.", "Coming soon: Nov 13.", "Coming soon: Nov 19.",
        "Coming soon: Nov 8.", "Coming soon: Oct 29.", "Coming soon: Dec 15.", "Coming soon: Oct 30.",
        "Coming soon: Dec 9.", "Coming soon: Dec 20.", "Coming soon: Dec 13.", "Coming soon: Dec 23.",
        "Coming soon: Nov 30.", "Coming soon: Dec 1.", "Coming soon: Nov 5.", "Coming soon: Nov 12.",
        "Coming soon: Nov 25.", "Coming soon: Nov 9."],
}

def change_status(row):
    for status_new, values in values_status.items():
        if row in values:
            return status_new
    return "unknown"

# заменим значения на созданные нами обобщающие статусы
df['status'] = df['status'].apply(change_status)

In [30]:
# снова проверяем количество уникальныз значений
uniq_status_sum = df['status'].nunique()
print(f'Количество уникальных значений столбца status: {uniq_status_sum}')

Количество уникальных значений столбца status: 15


Столбец propertyType

In [31]:
# посмотрим содержание столбца
uniq_propertyType_sum = df['propertyType'].nunique()
print(f'Количество уникальных значений столбца propertyType: {uniq_propertyType_sum}')

df['propertyType'].unique()[:50]

Количество уникальных значений столбца propertyType: 1280


array(['Single Family Home', 'single-family home', 'lot/land',
       'townhouse', 'Florida', nan, 'Single Family', 'coop', 'English',
       '2 Story', 'Townhouse', 'multi-family', 'Penthouse, Split-Level',
       'Multi-Family Home', 'Condo', 'condo', 'Land',
       'Condo/Townhome/Row Home/Co-Op', ' ', 'Detached, Two Story',
       '1 Story, Ranch', 'Other Style', 'Colonial', 'Transitional',
       'High Rise', 'mobile/manufactured',
       'Tri-Level, Northwestern Contemporary', 'Detached, One Story',
       'Craftsman', 'Single Detached, French', '1 Story, Traditional',
       'Single Detached, Traditional', 'Federal', 'Multi Family',
       'apartment', 'Traditional', 'Custom', 'Cooperative',
       'Contemporary/Modern, Traditional',
       'Cape Cod, Contemporary, Florida, Key West', 'Single Detached',
       'Mobile / Manufactured', 'Contemporary/Modern', 'Miscellaneous',
       'Mfd/Mobile Home', 'Bungalow', '1 Story', 'Spanish/Mediterranean',
       'Contemporary', 'Multi-Le

In [32]:
# смотрим сколько пропусков в столбце
df['propertyType'].isnull().sum()

34733

Объединим схожие значения доминирующих типов, остальные заменим на other

In [33]:
# сначала сократим количество вариантов названий, приведя все в нижний регистр
df['propertyType'] = df['propertyType'].str.lower()

# заменим похожие названия, выделила 9 основных типов, все остальные будут unknown
df['propertyType'] = df['propertyType'].replace(['single-family home', 'single family home', 'singlefamilyresidence', '1 story',
                                                'one story traditional', '1 story traditional', 'detached, one story',
                                                'single detached', 'single wide', 'single-wide mobile with land',
                                                '1 story, contemporary', '1 story, other (see remarks)',
                                                'single detached, french', '1 story, traditional', 'single detached, traditional',
                                                'one story', 'one level unit', '1 1/2 story', 'single wide mh',
                                                '1 story,traditional', '1 story, historic/older, traditional', '1 story, split level'],
                                        'single family')
df['propertyType'] = df['propertyType'].replace(['multi-family', 'multi-family home', 'duplex', 'triplex', 'fourplex',
                                                 'detached, two story', '2 story, other (see remarks)',
                                                 'multi_level', '2-story', 'two story', 'multi-level, modern', '2 stories, traditional',
                                                 '2 stories', 'traditional', 'attached or 1/2 duplex, traditional'],
                                        'multi family')
df['propertyType'] = df['propertyType'].replace(['coop', 'cooperative', 'condo/townhome/row home/co-op', 'condo/townhome','condominium',
                                                 'condo/unit', 'apartment/condo/townhouse', 'co-op', '2 story condo', 'high rise',
                                                 '2 unit condo', 'condo/townhome, contemporary/modern, loft, traditional',
                                                 'condo/townhome, hi-rise, resort property, vacation home, contemporary/modern',
                                                 'condo/townhome, other (see remarks)', 'condo/townhome, french',
                                                 'condo/townhome, hi-rise, contemporary/modern, loft', 'condo, other (see remarks)',
                                                 'condo/townhome, craftsman, traditional'],
                                        'condo')
df['propertyType'] = df['propertyType'].replace(['townhome style', 'townhouse-interior', 'townhouse-end unit',
                                                 'townhouse, attached/row', 'townhouse, northwestern contemporary', 'attached, townhouse',
                                                 'townhouse, 2-story', 'townhouse, two story, traditional', 'townhouse, villa'],
                                        'townhouse')
df['propertyType'] = df['propertyType'].replace(['lot/land'],
                                        'land')
df['propertyType'] = df['propertyType'].replace(['mobile/manufactured', 'mobile / manufactured', 'manufactured house', 'mfd/mobile home',
                                                 'manufactured home', 'manufactured double-wide', 'manufactured single-wide',
                                                 'mobile home 1 story', 'mobile manu - double wide','manufactured house,ranch, one story',
                                                 'manufactured house, ranch, one story, manufactured home',
                                                 'manufactured house, traditional, manufactured home', 'manufactured house, manufactured home',
                                                 '1 story, manufactured home - single wide', 'manufactured home, mobile home, ranch'],
                                        'mobile home')
df['propertyType'] = df['propertyType'].replace(['designated historical home', 'historical/conservation district', 'historic/older',
                                                 'historic vintage', 'historic', '1 story, historic/older, craftsman',
                                                 'historical/conservation district, single detached, contemporary/modern, traditional',
                                                 '1 story, historic/older, traditional, craftsman', 'historical, traditional',
                                                 '2 stories, historic/older, craftsman'],
                                        'historical')
df['propertyType'] = df['propertyType'].replace(['rancher', '1 story/ranch', '1 story, ranch', 'rancher, raised ranch', 'farms/ranches',
                                                 'hi ranch', 'ranch, one story', 'ranch, traditional', 'farm house',
                                                 'ranch, one story, duplex', 'farm house', 'ranch, transitional',
                                                 'bungalow, contemporary, ranch, traditional', '1 story, ranch, traditional, texas hill country',
                                                 '1 story, contemporary, ranch, historic/older, traditional', 'farm house, transitional',
                                                 'ranch, traditional, transitional', 'old world, ranch', 'ranch, split level', 
                                                 'ranch, spanish', 'farm/ranch house, single detached, contemporary/modern, ranch',
                                                 '2 stories, colonial, ranch', '1 story, ranch, craftsman', 'raised ranch, rancher'],
                                        'ranch')
df['propertyType'] = df['propertyType'].replace(['condominium (single level)', 'high-rise', 'mid-rise', 'low-rise (1-3 stories)',
                                                 'Flats', 'studio'],
                                        'apartment')
df['propertyType'] = df['propertyType'].replace('yes','other')

# заменим пропуски на other
df['propertyType'].fillna('other', inplace=True)

# основные типы
type_main = ['single family','multi family', 'condo', 'townhouse', 'land', 'mobile home', 'historical', 'ranch', 'apartment', 'unknown']
# функция проверяет, есть ли значение столбца в списке основных типов, если нет, то other
def change_type(type):
    if type in type_main:
        return type    # значение найдено в списке, остается без изменений
    else:
        return 'other'

# применим функцию
df['propertyType'] = df['propertyType'].apply(change_type)

# проверим уникальные значения и пропуски
uniq_propertyType_sum = df['propertyType'].nunique()
print(f'Количество уникальных значений столбца propertyType: {uniq_propertyType_sum}')

df['propertyType'].isnull().mean()*100

Количество уникальных значений столбца propertyType: 11


0.0

### Пропуски столбец street и zipcode

Столбец street

In [34]:
df_empty_street = df.loc[df['street'].isnull()]
print(df_empty_street)

          status propertyType street  baths  \
109863  For sale         land    NaN    0.0   
170142  For sale         land    NaN    0.0   

                                                homeFacts       city  \
109863  {'atAGlanceFacts': [{'factValue': '1996', 'fac...     Austin   
170142  {'atAGlanceFacts': [{'factValue': '', 'factLab...  Las Vegas   

                                                  schools  sqft zipcode state  \
109863  [{'rating': ['9/10', '10/10', '10/10'], 'data'...  5032   78746    TX   
170142  [{'rating': ['5/10', 'None/10', '5/10'], 'data...     0   89118    NV   

            target  private_pool_final  beds_final  
109863    $499,900               False         5.0  
170142  $1,009,091               False         0.0  


Так как для нахождения названия улицы у нас не хватает данных (по индексу, городу может быть несколько вариантов улиц), то строки с пропусками в данном столбце придется удалить.

In [35]:
# удаляем строки, где есть пропуски в столбце street
df = df.dropna(subset=['street'])

Столбец zipcode

Так как в столбце не обнаружено пропусков, проверим на скрытые пропуски.<br>
Найдены такие значения как '--', '0', '00000', а так же двойные индесы, через дефис.

In [36]:
# проверим содержание столбца zipcode
df['zipcode'].unique()

# отфильтруем некорректные значения найденных индексов
row_to_drop = df[(df['zipcode']=='--') | (df['zipcode']=='0')| (df['zipcode']=='00000')].index

# удалим строки
df = df.drop(row_to_drop)

# уберем вторую часть индекса после дефиса
df['zipcode'] = df['zipcode'].str.replace("-.+",'', regex=True)

### Изучение столбцов homeFacts и schools

Столбец schools

In [37]:
df['schools'].unique()

array(['[{\'rating\': [\'4\', \'4\', \'7\', \'NR\', \'4\', \'7\', \'NR\', \'NR\'], \'data\': {\'Distance\': [\'2.7 mi\', \'3.6 mi\', \'5.1 mi\', \'4.0 mi\', \'10.5 mi\', \'12.6 mi\', \'2.7 mi\', \'3.1 mi\'], \'Grades\': [\'3–5\', \'6–8\', \'9–12\', \'PK–2\', \'6–8\', \'9–12\', \'PK–5\', \'K–12\']}, \'name\': [\'Southern Pines Elementary School\', \'Southern Middle School\', \'Pinecrest High School\', \'Southern Pines Primary School\', "Crain\'s Creek Middle School", \'Union Pines High School\', \'Episcopal Day Private School\', \'Calvary Christian Private School\']}]',
       "[{'rating': ['4/10', 'None/10', '4/10'], 'data': {'Distance': ['1.65mi', '1.32mi', '1.01mi'], 'Grades': ['9-12', '3-8', 'PK-8']}, 'name': ['East Valley High School&Extension', 'Eastvalley Middle School', 'Trentwood Elementary School']}]",
       "[{'rating': ['8/10', '4/10', '8/10'], 'data': {'Distance': ['1.19mi', '2.06mi', '2.63mi'], 'Grades': ['6-8', 'K-5', '9-12']}, 'name': ['Paul Revere Middle School', 'Bren

Основную ценность для нас в данном словаре представляет информация рейтинга школы и расстояние до школы. Создадим новые признаки из данной информации.

In [38]:
# выберем значение рейтинга
df['schools_new'] = df['schools'].str.findall(r"\brating': ([\s\S]+?), 'data\b")

# функция выбирает список числовых значений рейтинга из вложенного списка
def choose_rating_list(data):
    data_new = data[0]
    return data_new
# применяем функцию
df['schools_new'] = df['schools_new'].apply(choose_rating_list)

# оставляем только числовые значения
def change_rating(rating):
    rating = rating.replace('[', '').replace(']', '').replace("'", '').replace('/10', '')
    return rating
# применяем функцию
df['schools_new'] = df['schools_new'].apply(change_rating)

In [39]:
# найдем среднее значение рейтинга
def find_average_rating(rating):
    rating = rating.split(', ')
    find_rating = [float(num) for num in rating if num.isdigit()]
    average_rating = np.average(find_rating) if find_rating else -1
    return average_rating

# применяем функцию, создаем новый столбец со средним рейтингом    
df['schools_rating'] = df['schools_new'].apply(find_average_rating)

In [40]:
# рассчитаем минимальную дистанцию до школы
distance_min = df['schools'].str.findall(r"\bDistance': ([\s\S]+?), 'Grades\b") # выбираем расстояние
distance_min = distance_min.apply(lambda x: x[0]) # выбираем первый элемент из каждого списка
distance_min = distance_min.str.replace('[a-zA-Z]','', regex=True) # удаляем буквенные символы из каждой строки
distance_min = distance_min.str.findall(r'\b([0-9]+.[0-9]+)') # ищем все числа в каждой строке
distance_min = distance_min.apply(lambda x: [float(i) for i in x]) # преобразуем в float
school_distance_min = distance_min.apply(lambda x: -1 if len(x)==0 else min(x)) # находим мин. расстояние, если список пустой, то -1 

#Создадим признак school_distance_min
df['school_distance_min'] = school_distance_min

# удаляем отработанные столбцы
df.drop(['schools', 'schools_new'], axis=1, inplace=True)

Столбец homeFacts

In [41]:
df['homeFacts'].unique()[1]

"{'atAGlanceFacts': [{'factValue': '2019', 'factLabel': 'Year built'}, {'factValue': '', 'factLabel': 'Remodeled year'}, {'factValue': '', 'factLabel': 'Heating'}, {'factValue': '', 'factLabel': 'Cooling'}, {'factValue': '', 'factLabel': 'Parking'}, {'factValue': '5828 sqft', 'factLabel': 'lotsize'}, {'factValue': '$159/sqft', 'factLabel': 'Price/sqft'}]}"

Признак содержит информацию о годе постройки дома, реставрации дома, кондиционере, отоплении, паркинге, площади и цене за фут кв. Последнее значение 'Price/sqft' мы не можем использовать, потому что эти данные будут создавать утечку данных при прогнозировании цены.
Посмотрим, что сколько и каких данных в остальных лейблах.

In [42]:
# выберем значение Label и Value с помощью регулярных выражений
Label = df['homeFacts'].str.findall(r"\bfactLabel': ([\s\S]+?)[}\b]")
Value = df['homeFacts'].str.findall(r"\bfactValue': ([\s\S]+?), 'factLabel\b")

# создадим список новых признаков
label_list = ','.join(Label[0]).replace("'","").split(',')
label_list

['Year built',
 'Remodeled year',
 'Heating',
 'Cooling',
 'Parking',
 'lotsize',
 'Price/sqft']

In [43]:
# добавим признаки и их значения в датафрейм
for i, val in enumerate(label_list):
    df[val]=Value.apply(lambda x: x[i])

Столбец Year built

In [44]:
# займемся признаком Year built
df['Year built'].unique()

# удалим лишние кавычки
df['Year built'] = df['Year built'].str.replace("'",'', regex=True)

# в признаке есть "мусорные" значения: заменим их на 'unknown'
df['Year built'] = df['Year built'].str.replace('^\s*$','unknown', regex=True)
df['Year built'] = df['Year built'].str.replace('No Data','unknown')
df['Year built'] = df['Year built'].str.replace('559990649990','unknown')
df['Year built'] = df['Year built'].str.replace('^1$','unknown', regex=True)
df['Year built'] = df['Year built'].str.replace('None','unknown')

# так же есть странные значения годов: '1060', '1019', '1057', скорее всего это опечатка и имеются в виду 1900 года, заменим их тоже
df['Year built'] = df['Year built'].str.replace('1060','1960')
df['Year built'] = df['Year built'].str.replace('1019','1919')
df['Year built'] = df['Year built'].str.replace('1057','1957')

Столбец Remodeled year

In [45]:
# Займемся столбцом Remodeled year, заменим некорректные значения на None,
# далее сделаем столбец бинарным, где есть значения ставим True - реконструкция была, False - не было
df['Remodeled year'].unique() # "'0'""'''None'1111'

# удалим лишние кавычки и некорректные значения
df['Remodeled year'] = df['Remodeled year'].str.replace("'",'', regex=True)
df['Remodeled year'] = df['Remodeled year'].str.replace('^0$', 'None', regex=True)
df['Remodeled year'] = df['Remodeled year'].str.replace('1111', 'None', regex=True)
df['Remodeled year'] = df['Remodeled year'].str.replace('^\s*$','None', regex=True)

# функция определяет была ли реконструкция
def remodeled_bool(value):
    if isinstance(value, str) and re.match(r'^\d{4}$', value):
        return True
    else:
        return False
# применяем функцию
df['Remodeled year_final'] = df['Remodeled year'].apply(remodeled_bool)

Столбец Heating

In [46]:
# посмотрим содержание столбца
df['Heating'].str.lower().value_counts()

'forced air'                                                      134307
''                                                                105753
'other'                                                            29622
'electric'                                                         10216
'gas'                                                               9296
                                                                   ...  
'zoned heating, wall unit heating, forced air heating'                 1
'baseboard, spacewallunit'                                             1
'hot air, stove-pellet'                                                1
'natural gas, space heater'                                            1
'baseboard, hot water, programmable thermostat, radiant floor'         1
Name: Heating, Length: 1919, dtype: int64

Столбец содержит различные виды отопления, примерно четверть данных не заполненны. Будем считать, что там, где есть надписи, там есть отопление. Создадим булевый столбец, где есть отопление - True, где нет - False

In [47]:
# удалим лишние кавычки
df['Heating'] = df['Heating'].str.replace("'",'', regex=True)

# функция меняет столбец на булевые значения, там, где есть данные - True
df['Heating_final'] = df['Heating'].apply(lambda x: True if x not in ['', 'No Data', 'No Heat Fuel', 'No Heat', 'None'] else False)
df['Heating_final'].value_counts()

True     259154
False    117973
Name: Heating_final, dtype: int64

Столбец Cooling

In [48]:
# посмотрим содержание столбца
df['Cooling'].str.lower().value_counts()

'central'                                                                158743
''                                                                       120390
'central air'                                                             14384
'no data'                                                                 10615
'has cooling'                                                              9730
                                                                          ...  
'central gas, propane, zoned'                                                 1
'other (see remarks), panel/floor/wall, window unit'                          1
'multi units, zoned cooling'                                                  1
'central air, g-energy star hvac, gas hot air/furnace, multizone a/c'         1
'central a/c (gas), central heat (gas), heat pump'                            1
Name: Cooling, Length: 1439, dtype: int64

In [49]:
# уберем лишние кавычки и изучим подробнее содержание
df['Cooling'] = df['Cooling'].str.replace("'",'', regex=True)
df['Cooling'].unique()[200:205]

# проверим сколько пропущенных данных в %
cooling_nodata_per = round(((df[df['Cooling']==''].shape[0]) / (df['Cooling'].shape[0]) * 100), 1)
print(f'Пропущенные данные в столбце Cooling: {cooling_nodata_per} %')

Пропущенные данные в столбце Cooling: 31.9 %


Столбец Cooling содержит противоречвые данные, потому что в отличие от отопления (там были различные виды отопления), в столбце кондиционеры перечислены и кондиционеры, и отопление. К тому же в данном столбце пропусков почти 32%. Поэтому данный столбец придется удалить.

Столбец Parking

In [50]:
# уберем лишние кавычки и изучим подробнее содержание
df['Parking'] = df['Parking'].str.replace("'",'', regex=True)
df['Parking'].unique()[200:205]

# проверим сколько пропущенных данных в %
Parking_nodata_per = round(((df[df['Parking']==''].shape[0]) / (df['Parking'].shape[0]) * 100), 1)
print(f'Пропущенные данные в столбце Parking: {Parking_nodata_per} %')

Пропущенные данные в столбце Parking: 45.6 %


Пропущенных данных еще больше 45,6%. Восполнить данную информацию не представляется возможным. Удаляем данный столбец.

Столбец lotsize показывает площадь дома или участка, что для нас не несет полезной информации, удаляем его.<br>
Столбец Price/sqft создает утечку данных, удаляем его.

In [51]:
# удаляем лишние и отработанные столбцы
df = df.drop(['homeFacts', 'Remodeled year', 'Cooling', 'Parking', 'lotsize', 'Price/sqft', 'Heating'], axis=1)

### Обработка целевого признака target

In [52]:
# посомтрим содержание таргета
df['target'].unique()[0:10]

array(['$418,000', '$310,000', '$2,895,000', '$2,395,000', '$5,000',
       '$209,000', '181,500', '68,000', '$244,900', '$311,995'],
      dtype=object)

In [53]:
# проверим сколько пропусков в таргете в %
round((df['target'].isnull().mean()*100), 2)

0.66

В таргете есть пропуски, при этом строки, где нет таргета не могут нам помочь в модели регрессии, так как у нас не будет возможности оценить качество модели. Поэтому эти строки удаляем.

In [54]:
# удаляем строки, где нет таргета
df = df.dropna(subset=['target'])

In [55]:
# уберем лишние символы
df['target'] = df['target'].replace('\$', '', regex=True)
df['target'] = df['target'].replace(',', '', regex=True)
df['target'] = df['target'].replace('\+', '', regex=True)

Есть цена с пометкой /mo, скорее всего это цена в месяц. Проверим, какой статус у данных объектов.<br>
Так как цена в месяц нам, опять же, не подходит для оценки качества модели, то строки с такой ценой удаляем.

In [56]:
# проверим статус объектов с ценой /mo
df[df['target'].str.contains('/mo',regex=True)].head()

# статус таких квартир For rent
# удаляем такие квартиры
df = df[~df['target'].str.contains('/mo', regex=True)]

In [57]:
# переведем столбец в числовое значение
df['target'] = df['target'].astype(int)

In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 374249 entries, 0 to 377134
Data columns (total 16 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   status                374249 non-null  object 
 1   propertyType          374249 non-null  object 
 2   street                374249 non-null  object 
 3   baths                 374249 non-null  float64
 4   city                  374249 non-null  object 
 5   sqft                  374249 non-null  int32  
 6   zipcode               374249 non-null  object 
 7   state                 374249 non-null  object 
 8   target                374249 non-null  int32  
 9   private_pool_final    374249 non-null  bool   
 10  beds_final            374249 non-null  float64
 11  schools_rating        374249 non-null  float64
 12  school_distance_min   374249 non-null  float64
 13  Year built            374249 non-null  object 
 14  Remodeled year_final  374249 non-null  bool   
 15  

## *Выводы 1 части работы:*
- изучили данные
- нашли и избавились от пропусков в данных
- сократили количество столбцов, избавились от повторяющихся данных
- избавились от избыточных данных
- данные сохраняем в файл, чтоб далее было удобнее работать

In [64]:
# сохраняем предобработанные данные в CSV-файл для упрощения дальнейшей работы
df.to_csv("data/cleaned_data.csv", index=False)